In [32]:
import numpy as np
import pandas as pd

import re

In [4]:
df_inbox = pd.read_json (r'.\data\inbox.json')
df_inbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н..."
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке..."
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...
4,Ustinov.A.Yury@omega.sbrf.ru,Ustinov.A.Yury@omega.sbrf.ru,[Smirnov.I.Yurye@omega.sbrf.ru],"[Konyrev.D.An@sberbank.ru, Tsutsiev.D.E@omega....",2019-12-30 13:28:29+00:00,RE: Построение модели извлечения,<960e189c9be64d4b80e9bc2cb84a6523@omega.sbrf.ru>,"Иван, добрый вечер!\r\n\r\nХотелось бы уточнит...","[{'text': 'Иван, добрый вечер! Хотелось бы уто..."


In [10]:
df_inbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 9 columns):
owner         3843 non-null object
from          3843 non-null object
to            3843 non-null object
cc            3843 non-null object
date          3843 non-null datetime64[ns, UTC]
subject       3806 non-null object
message_id    3843 non-null object
text          3843 non-null object
forward       3827 non-null object
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 270.3+ KB


In [5]:
df_inbox['owner'].value_counts()

moy_sberbank@sberbank.ru          394
Yusupov.V.A@omega.sbrf.ru         379
Shtuntsayger.I.Ef@sberbank.ru     182
MYUKudryavtcev@sberbank.ru        173
Burlakova.M.Val@sberbank.ru       173
                                 ... 
Troshnev.N.N@sberbank.ru            1
AAPopov1@sberbank.ru                1
Kovalchuk.A.Alekse@sberbank.ru      1
Blinova.A.Ol@omega.sbrf.ru          1
eugene.rudaev@abbyy.com             1
Name: owner, Length: 262, dtype: int64

In [6]:
df_inbox['owner'].nunique()

262

In [12]:
df_outbox = pd.read_json (r'.\data\outbox.json')
df_outbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ..."
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф..."
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про..."
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р..."
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к..."


In [13]:
set(df_inbox['message_id']) & set(df_outbox['message_id'])

{'<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>'}

In [24]:
list(df_inbox[df_inbox['message_id']=='<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>']['text'])

['Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, который появился в пн был устранен, сборка поставлена на ИФТ.\r\n\r\n\r\n\r\nhttp://sbrf-nexus.ca.sbrf.ru/nexus/content/repositories/Nexus_PROD/Nexus_PROD/CI02029517_ML_CDS/cds/01.001.00-55/cds-01.001.00-55-distrib.zip\r\n\r\n\r\n\r\nПросьба поставить сборку на ПРОМ.\r\n\r\n\r\n\r\nС уважением,\r\n\r\nКонырев Дмитрий\r\n\r\n']

In [23]:
list(df_outbox[df_outbox['message_id']=='<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>']['text'])

['Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, который появился в пн был устранен, сборка поставлена на ИФТ.\r\n\r\n\r\n\r\nhttp://sbrf-nexus.ca.sbrf.ru/nexus/content/repositories/Nexus_PROD/Nexus_PROD/CI02029517_ML_CDS/cds/01.001.00-55/cds-01.001.00-55-distrib.zip\r\n\r\n\r\n\r\nПросьба поставить сборку на ПРОМ.\r\n\r\n\r\n\r\nС уважением,\r\n\r\nКонырев Дмитрий\r\n\r\n']

In [25]:
df_outbox['owner'].value_counts()

Konyrev.D.An@sberbank.ru    768
Name: owner, dtype: int64

In [30]:
df_inbox[df_inbox['from']=='Konyrev.D.An@sberbank.ru']

,owner,from,to,cc,date,subject,message_id,text,forward
1070,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Bagnenko.V.V@sberbank.ru, Alekseev.D.Genna@om...","[Timofeev.A.Valery@omega.sbrf.ru, Shtuntsayger...",2019-10-09 13:33:12+00:00,Устранение дефекта сервисов CDS ТД,<7e5f52332ea04b14ad61a1b07b42db9d@sberbank.ru>,"Коллеги, добрый день.\r\n\r\n\r\n\r\nДефект, к...","[[Коллеги, добрый день.\nДефект, который появи..."


In [27]:
df_outbox[df_outbox['owner']=='Konyrev.D.An@sberbank.ru']

,owner,from,to,cc,date,subject,message_id,text,forward
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ..."
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф..."
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про..."
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р..."
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к..."
...,...,...,...,...,...,...,...,...,...
763,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Karpets.M.Val@omega.sbrf.ru],[],2019-01-11 11:32:07+00:00,None,<075292f91f9d4e6998d6d7165326a40a@CAB-VSP-MBX1...,\r\n,"[[\n, 0, {'text': ' '}]]"
764,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Karpets.M.Val@omega.sbrf.ru],[],2019-01-10 12:27:40+00:00,None,<2a9cb95ffc234def93861d5306c5f12c@CAB-VSP-MBX1...,\r\n,"[[\n, 0, {'text': ' '}]]"
765,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[OUT-Dmitrieva-AA@mail.ca.sbrf.ru],[],2019-01-10 08:47:57+00:00,FW: Наиболее актуальная версия Глоссария,<05b523ac07214903b809e33b10c8c9fe@CAB-VSP-MBX1...,\r\n\r\nFrom: Бурлакова Марина Валериевна <Bur...,"[{'text': ' '}, {'to': ['Конырев Дмитрий Андре..."
766,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Protopopov.V.Al@omega.sbrf.ru, Protopopov.V.A...",[],2019-01-09 15:38:41+00:00,Последний отчет,<8a434a7eae854b8590d39bfc7d1883ce@CAB-VSP-MBX1...,"Добрый день,\r\n\r\nВысылаю в письме отчет по ...","[[Добрый день,\nВысылаю в письме отчет по сущн..."


In [31]:
df_outbox['to'].value_counts()

[Konyrev.D.A@mail.ca.sbrf.ru]                                                                                                                                           66
[MYUKudryavtcev@sberbank.ru]                                                                                                                                            60
[Yusupov.V.A@omega.sbrf.ru]                                                                                                                                             35
[Burlakova.M.Val@sberbank.ru]                                                                                                                                           25
[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@omega.sbrf.ru]                                                                                                               21
                                                                                                                                                 

In [46]:
def search_pattern(pattern, input_to):
    #pattern = "Konyrev.D.An@sberbank.ru"
    Flag = False
    
    for input_txt in input_to:
        lst = re.findall(pattern, input_txt)
        if len(lst)>0:
            Flag=True
            break

    return Flag

In [33]:
list(df_inbox['to'])

[['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An@sberbank.ru'],
 ['Konyrev.D.An@sberbank.ru', 'Fedotov.I.Ser@omega.sbrf.ru'],
 ['Gaparkhoev.V.L@omega.sbrf.ru', 'Konyrev.D.An@sberbank.ru'],
 ['Konyrev.D.An@sberbank.ru'],
 ['Smirnov.I.Yurye@omega.sbrf.ru'],
 ['Ulyanov.A.A@sberbank.ru',
  'Ocheretny.A.S@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Shavrina.T.O@sberbank.ru'],
 ['Ocheretny.A.S@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru'],
 ['moy_sberbank@sberbank.ru'],
 ['NRZelinskiy@sberbank.ru',
  'Ocheretny.A.S@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['Ocheretny.A.S@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['AVStrelnikov@sberbank.ru',
  'Shtuntsayger.I.Ef@sberbank.ru',
  'AVS

In [48]:
df_inbox['Konyrev'] = np.vectorize(search_pattern)('Konyrev.D.An@sberbank.ru', df_inbox['to'])

In [49]:
Konyrev_inbox = df_inbox[df_inbox['Konyrev']==True]

In [50]:
Konyrev_inbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2189 entries, 0 to 3842
Data columns (total 10 columns):
owner         2189 non-null object
from          2189 non-null object
to            2189 non-null object
cc            2189 non-null object
date          2189 non-null datetime64[ns, UTC]
subject       2153 non-null object
message_id    2189 non-null object
text          2189 non-null object
forward       2177 non-null object
Konyrev       2189 non-null bool
dtypes: bool(1), datetime64[ns, UTC](1), object(8)
memory usage: 173.2+ KB


In [51]:
Konyrev_inbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev
0,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:51:45+00:00,RE: IRPM-202 отсутствует инструкция по установ...,<a1562077a5ff40249595581be759d4f5@omega.sbrf.ru>,"Просьба приложить это в виде инструкции (""рыбу...",[{'text': 'Просьба приложить это в виде инстру...,True
1,Gaparkhoev.V.L@omega.sbrf.ru,Gaparkhoev.V.L@omega.sbrf.ru,"[Konyrev.D.An@sberbank.ru, Fedotov.I.Ser@omega...","[Pinchuk-LAL@mail.ca.sbrf.ru, support_ird@mail...",2019-12-31 09:39:55+00:00,Re: IRPM-202 отсутствует инструкция по установ...,<1577785193.3767.0.camel@omega.sbrf.ru>,"Добрый день, Илья!\r\nМодели и квоты не меняли...","[{'text': 'Добрый день, Илья! Модели и квоты н...",True
2,Fedotov.I.Ser@omega.sbrf.ru,Fedotov.I.Ser@omega.sbrf.ru,"[Gaparkhoev.V.L@omega.sbrf.ru, Konyrev.D.An@sb...","[AVSipiev@sberbank.ru, Bagnenko.V.V@sberbank.r...",2019-12-31 09:11:08+00:00,IRPM-202 отсутствует инструкция по установке н...,<37eaa2b19f3249bcb211646d4a1a9bdc@omega.sbrf.ru>,"Коллеги, добрый день!\r\n\r\n\r\n\r\nПри устан...","[{'text': 'Коллеги, добрый день! При установке...",True
3,portal@aclweb.org,portal@aclweb.org,[Konyrev.D.An@sberbank.ru],[],2019-12-30 14:31:10+00:00,[ACL Portal] Newsletter 2019,<e310dfa11be393364b4403f537f38d52@aclweb.org>,ВНЕШНЯЯ ПОЧТА: Если вы не знаете отпр...,[[ВНЕШНЯЯ ПОЧТА: Если вы не знаете от...,True
5,Krivobok.M.A@sberbank.ru,Krivobok.M.A@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Ocheretny.A.S@sberba...",[AChertok@sberbank.ru],2019-12-30 12:12:04+00:00,RE: материалы со встречи по DS итогам 2019,<67d07ae64f014007b59ab01de9f2c73c@sberbank.ru>,"Коллеги, привет!\r\nОбновлен слайд по TagMe.\r...","[{'text': 'Коллеги, привет! Обновлен слайд по ...",True


In [53]:
df_outbox['Ocheretny'] = np.vectorize(search_pattern)('Ocheretny.A.S@sberbank.ru', df_outbox['to'])
df_outbox.head()

,owner,from,to,cc,date,subject,message_id,text,forward,Ocheretny
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ...",True
1,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],"[MYUKudryavtcev@sberbank.ru, Shavrina.T.O@sber...",2019-12-27 08:11:06+00:00,RE: LJ-образец.sql,<ad52878dc4f945d79c18bbe135026f86@sberbank.ru>,"Андрей, привет!\r\n\r\nДовольно забавный форма...","[{'text': 'Андрей, привет! Довольно забавный ф...",False
2,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[AChertok@sberbank.ru],[],2019-12-24 14:23:51+00:00,Сервер с GPU для Башен,<c4229bc0b1784748a426bad0d08177c1@sberbank.ru>,"Андрей, привет!\r\n\r\nВ продолжении разговора...","[[Андрей, привет!\nВ продолжении разговора про...",False
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р...",True
4,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Burlakova.M.Val@sberbank.ru, Konyrev.D.An@ome...",[Protopopov.V.Al@omega.sbrf.ru],2019-12-24 12:46:18+00:00,RE: Оценка правовых рисков,<f317433a695740f1b1d54c71e04307bc@sberbank.ru>,"Добрый день!\r\n\r\nСпасибо, мы сейчас с коман...","[{'text': 'Добрый день! Спасибо, мы сейчас с к...",False


In [62]:
#Конырев пишет Ocheretny
Ocheretny_outbox = df_outbox[df_outbox['Ocheretny']==True]
Ocheretny_outbox.sort_values(by='date')

,owner,from,to,cc,date,subject,message_id,text,forward,Ocheretny
285,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-09-10 15:36:57+00:00,Примеры ГПХ,<ecb6fb1f16ae4f688158cb498fd57cf7@sberbank.ru>,"Добрый день!\r\n\r\nКак и обещал, высылаю прим...","[[Добрый день!\nКак и обещал, высылаю примеры ...",True
153,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 08:05:02+00:00,RE: Выступление на радио Медиаметрикс,<89c7825e5199476f90aad83f398a6753@sberbank.ru>,"Добрый день!\r\n\r\nПо поводу Академии ИИ, дей...","[{'text': 'Добрый день! По поводу Академии ИИ,...",True
151,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 12:58:04+00:00,RE: Выступление на радио Медиаметрикс,<f23f496c8fad4f41bbbb32690c83c496@sberbank.ru>,"Добрый день, прошу прощения.\r\n\r\nРуководите...","[{'text': 'Добрый день, прошу прощения. Руково...",True
150,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-16 13:35:38+00:00,RE: Выступление на радио Медиаметрикс,<e0abb98c695a4a9eb441e2772865119b@sberbank.ru>,"К моему сожалению нет.\r\n\r\nС уважением,\r\n...","[{'text': 'К моему сожалению нет. С уважением,...",True
149,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[kondratieva@vbudushee.ru, satikova@vbudushee....",[],2019-10-17 14:39:57+00:00,RE: Выступление на радио Медиаметрикс,<dfc571cdfe5b4186bc6a313758ecc3e0@sberbank.ru>,Добрый день!\r\n\r\nХотелось бы добавить вопро...,[{'text': 'Добрый день! Хотелось бы добавить в...,True
3,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,"[Ocheretny.A.S@sberbank.ru, Ocheretny.A.S@omeg...",[],2019-12-24 12:59:47+00:00,Роли в команде,<08b6ad91178543f7b4aada2eab27fe1f@sberbank.ru>,"Добрый день,\r\n\r\nКак и договаривались высыл...","[[Добрый день,\nКак и договаривались высылаю р...",True
0,Konyrev.D.An@sberbank.ru,Konyrev.D.An@sberbank.ru,[Ocheretny.A.S@sberbank.ru],[],2019-12-27 11:09:47+00:00,FW: DS - итоги 2019,<f41b3f2d974c4117be1ec9139b6f779d@sberbank.ru>,"Андрей, привет!\r\n\r\nСделано, смотри.\r\n\r\...","[{'text': 'Андрей, привет! Сделано, смотри. С ...",True


In [67]:
Ocheretny_outbox['forward'].to_list()[0]

[{'text': 'Андрей, привет!\nСделано, смотри.\nС уважением,\nКонырев Дмитрий\n'},
 {'to': ['Ульянов Александр Анатольевич ',
   'Ulyanov.A.A@sberbank.ru',
   ' Шаврина Татьяна Олеговна ',
   'Shavrina.T.O@sberbank.ru',
   ' Кривобок Максим Анатольевич ',
   'Krivobok.M.A@sberbank.ru',
   ' Зелинский Никита Ростиславович ',
   'NRZelinskiy@sberbank.ru',
   ' Темин Лев Владимирович ',
   'LVTemin@sberbank.ru',
   ' Конырев Дмитрий Андреевич ',
   'Konyrev.D.An@sberbank.ru'],
  'from': ['Черток Андрей Викторович ', 'AChertok@sberbank.ru'],
  'cc': ['Еременко Максим Алексеевич ',
   'MAEremenko@sberbank.ru',
   ' Кудрявцев Михаил Юрьевич ',
   'MYUKudryavtcev@sberbank.ru',
   ' Очеретный Андрей Сергеевич ',
   'Ocheretny.A.S@sberbank.ru'],
  'date': '2019-12-25T20:42:00',
  'subject': 'DS - итоги 2019',
  'text': 'Коллеги, всем привет.\nЗа 2019 год офисом CDS было реализовано много проектов, но не о всех из них мы информировали о них руководство и коллег.\nПредлагаем в пятницу подвести итог

In [55]:
Ocheretny_outbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 285
Data columns (total 10 columns):
owner         7 non-null object
from          7 non-null object
to            7 non-null object
cc            7 non-null object
date          7 non-null datetime64[ns, UTC]
subject       7 non-null object
message_id    7 non-null object
text          7 non-null object
forward       7 non-null object
Ocheretny     7 non-null bool
dtypes: bool(1), datetime64[ns, UTC](1), object(8)
memory usage: 567.0+ bytes


In [59]:
#Ocheretny пишет Коныреву
list(df_inbox[df_inbox['owner']=='Ocheretny.A.S@sberbank.ru']['to'])

[['NRZelinskiy@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Krivobok.M.A@sberbank.ru'],
 ['Fenogenova.A.S@sberbank.ru',
  'Ulyanov.A.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'NRZelinskiy@sberbank.ru'],
 ['Ulyanov.A.A@sberbank.ru',
  'Shavrina.T.O@sberbank.ru',
  'Krivobok.M.A@sberbank.ru',
  'NRZelinskiy@sberbank.ru',
  'LVTemin@sberbank.ru',
  'Konyrev.D.An@sberbank.ru',
  'Fenogenova.A.S@sberbank.ru'],
 ['satikova@vbudushee.ru', 'Konyrev.D.An@sberbank.ru']]

In [61]:
df_inbox[df_inbox['owner']=='Ocheretny.A.S@sberbank.ru'].sort_values(by='date')

,owner,from,to,cc,date,subject,message_id,text,forward,Konyrev
990,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[satikova@vbudushee.ru, Konyrev.D.An@sberbank.ru]",[],2019-10-14 14:48:38+00:00,RE: Выступление на радио Медиаметрикс,<278b83c4eda14d5b9f1f2d4f194f0568@sberbank.ru>,"Дария, привет.\r\n\r\nСегодня обсудили с Димо...","[{'text': 'Дария, привет. Сегодня обсудили с Д...",True
24,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[Ulyanov.A.A@sberbank.ru, Shavrina.T.O@sberban...",[AChertok@sberbank.ru],2019-12-27 14:06:00+00:00,материалы со встречи по DS итогам 2019,<a129d958fef44e0f9b6e4682a8c700b3@sberbank.ru>,По итогам жду короткое описание от каждой кома...,[[По итогам жду короткое описание от каждой ко...,True
16,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[Fenogenova.A.S@sberbank.ru, Ulyanov.A.A@sberb...",[AChertok@sberbank.ru],2019-12-27 21:37:14+00:00,RE: материалы со встречи по DS итогам 2019,<04de50ea70804c3cb7f89c9279a30199@sberbank.ru>,"Коллеги,\r\n\r\nВ приложении сборка, в которую...","[{'text': 'Коллеги, В приложении сборка, в кот...",True
14,Ocheretny.A.S@sberbank.ru,Ocheretny.A.S@sberbank.ru,"[NRZelinskiy@sberbank.ru, Ulyanov.A.A@sberbank...","[AChertok@sberbank.ru, Shavrina.T.O@sberbank.ru]",2019-12-28 15:25:30+00:00,RE: материалы со встречи по DS итогам 2019,<f9a85ed6289c41b8a4d686588689aaa9@sberbank.ru>,"Коллеги, привет.\r\n\r\nВ приложении новая пер...","[{'text': 'Коллеги, привет. В приложении новая...",True
